In [71]:
import pandas as pd

In [72]:
# --- 1. Cargar los CSV ---
epc1 = pd.read_csv("C:/Users/juana/Desktop/liquideuda_project/datasets_clean/EPC1_clean.csv")
epc2 = pd.read_csv("C:/Users/juana/Desktop/liquideuda_project/datasets_clean/EPC2_clean.csv")
epc3 = pd.read_csv("C:/Users/juana/Desktop/liquideuda_project/datasets_clean/EPC3_clean.csv")
epc4 = pd.read_csv("C:/Users/juana/Desktop/liquideuda_project/datasets_clean/EPC4_clean.csv")
epc5 = pd.read_csv("C:/Users/juana/Desktop/liquideuda_project/datasets_clean/EPC5_clean.csv")
epc6 = pd.read_csv("C:/Users/juana/Desktop/liquideuda_project/datasets_clean/EPC6_clean.csv")

In [73]:
# --- 2. Eliminar la columna 'total' en todas ---
for df in [epc1, epc2, epc3, epc4, epc5, epc6]:
    if "total" in df.columns:
        df.drop(columns=["total"], inplace=True)

# --- 2b. Agrupar epc2 para sumar físicas + empresas ---
epc2 = (
    epc2.groupby(["id_geo___ca", "comunidad_provincia", "año", "trimestre"], as_index=False)
        .sum(numeric_only=True)
)


In [74]:

# --- 3. Unir todas las tablas progresivamente ---
df_merged = epc1.merge(epc2, on=["id_geo___ca","comunidad_provincia","año","trimestre"], how="outer") \
                .merge(epc3, on=["id_geo___ca","comunidad_provincia","año","trimestre"], how="outer") \
                .merge(epc4, on=["id_geo___ca","comunidad_provincia","año","trimestre"], how="outer") \
                .merge(epc5, on=["id_geo___ca","comunidad_provincia","año","trimestre"], how="outer") \
                .merge(epc6, on=["id_geo___ca","comunidad_provincia","año","trimestre"], how="outer")

# --- 4. Verificar resultado ---
print("Dimensiones finales:", df_merged.shape)
print(df_merged[["id_geo___ca","comunidad_provincia","año","trimestre"]].drop_duplicates().shape[0], "filas únicas")
print(df_merged.head())

Dimensiones finales: (882, 56)
882 filas únicas
   id_geo___ca comunidad_provincia  s_limitadas  s_anonimas  sotras  \
0          104             Almería          8.0         0.0     0.0   
1          104             Almería         10.0         0.0     0.0   
2          104             Almería         10.0         0.0     0.0   
3          104             Almería         11.0         0.0     1.0   
4          104             Almería          8.0         0.0     0.0   

   personas_fisicas   año  trimestre  t_voluntario  t_necesario  ...  \
0              14.0  2021          1           9.0          0.0  ...   
1              21.0  2021          2          10.0          0.0  ...   
2              19.0  2021          3          12.0          0.0  ...   
3              26.0  2021          4          17.0          0.0  ...   
4              15.0  2022          1          11.0          0.0  ...   

   v_entre_2_y_5  v_entre_5_y_10  v_10_o_mas  sd_y  a_hasta_4  a_entre_5_y_8  \
0           

In [75]:
# Ver cuántas filas totales hay
print("Total de filas después del merge:", len(df_merged))

# Ver cuántas combinaciones únicas de clave hay
unique_keys = df_merged.drop_duplicates(subset=["id_geo___ca", "comunidad_provincia", "año", "trimestre"])
print("Filas únicas por clave:", len(unique_keys))

# Comparar
if len(df_merged) == len(unique_keys):
    print("✅ No hay duplicados en las claves, el merge está correcto.")
else:
    print("⚠️ Hay duplicados: el merge generó filas de más.")
    # Opcional: ver ejemplos de duplicados
    dupes = df_merged[df_merged.duplicated(subset=["id_geo___ca", "comunidad_provincia", "año", "trimestre"], keep=False)]
    print(dupes.head())


Total de filas después del merge: 882
Filas únicas por clave: 882
✅ No hay duplicados en las claves, el merge está correcto.


In [76]:
# --- 5. Validar vacíos en cada columna ---
nulls = df_merged.isna().sum().reset_index()
nulls.columns = ["columna", "nulos"]
nulls["%_nulos"] = (nulls["nulos"] / len(df_merged) * 100).round(2)

# Mostrar las primeras filas ordenadas por más vacíos
print(nulls.sort_values("%_nulos", ascending=False).head(20))




             columna  nulos  %_nulos
2        s_limitadas      4     0.45
3         s_anonimas      4     0.45
4             sotras      4     0.45
28                 o      2     0.23
43  v_entre_025_y_05      2     0.23
32                 s      2     0.23
33               na0      2     0.23
34             na1_2      2     0.23
35             na3_5      2     0.23
36             na6_9      2     0.23
37           na10_19      2     0.23
38           na20_49      2     0.23
39           na50_99      2     0.23
40       na100_o_mas      2     0.23
41              sd_x      2     0.23
42    v_menos_de_025      2     0.23
45     v_entre_1_y_2      2     0.23
44    v_entre_05_y_1      2     0.23
30                 q      2     0.23
46     v_entre_2_y_5      2     0.23


In [77]:
print("Duplicados exactos:", df_merged.duplicated().sum())


Duplicados exactos: 0


In [78]:
print(df_merged.dtypes)


id_geo___ca              int64
comunidad_provincia     object
s_limitadas            float64
s_anonimas             float64
sotras                 float64
personas_fisicas       float64
año                      int64
trimestre                int64
t_voluntario           float64
t_necesario            float64
t_consecutivo          float64
c_ordinario            float64
c_especial             float64
c_sin_masa             float64
a                      float64
b                      float64
c                      float64
d                      float64
e                      float64
f                      float64
g                      float64
h                      float64
i                      float64
j                      float64
k                      float64
l                      float64
m                      float64
n                      float64
o                      float64
p                      float64
q                      float64
r                      float64
s       

In [79]:
print(df_merged.describe())

       id_geo___ca  s_limitadas  s_anonimas      sotras  personas_fisicas  \
count   882.000000   878.000000  878.000000  878.000000        882.000000   
mean    799.122449    24.462415    1.502278    0.689066        120.575964   
std     488.796872    55.759525    3.833403    1.460274        269.879789   
min     104.000000     0.000000    0.000000    0.000000          0.000000   
25%     407.000000     4.000000    0.000000    0.000000         13.000000   
50%     747.000000     9.000000    0.000000    0.000000         40.000000   
75%    1106.000000    20.000000    1.000000    1.000000        101.750000   
max    1952.000000   423.000000   35.000000   10.000000       2801.000000   

               año   trimestre  t_voluntario  t_necesario  t_consecutivo  ...  \
count   882.000000  882.000000    882.000000   882.000000     882.000000  ...   
mean   2022.777778    2.388889    128.371882     0.680272      18.058957  ...   
std       1.315430    1.113130    301.887099     1.697911      

In [80]:
#cada entrada de provincia y periodo es unica
print(len(df_merged), len(df_merged.drop_duplicates(subset=["id_geo___ca","año","trimestre"])))


882 828


In [81]:
df_merged.columns

Index(['id_geo___ca', 'comunidad_provincia', 's_limitadas', 's_anonimas',
       'sotras', 'personas_fisicas', 'año', 'trimestre', 't_voluntario',
       't_necesario', 't_consecutivo', 'c_ordinario', 'c_especial',
       'c_sin_masa', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k',
       'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 'na0', 'na1_2', 'na3_5',
       'na6_9', 'na10_19', 'na20_49', 'na50_99', 'na100_o_mas', 'sd_x',
       'v_menos_de_025', 'v_entre_025_y_05', 'v_entre_05_y_1', 'v_entre_1_y_2',
       'v_entre_2_y_5', 'v_entre_5_y_10', 'v_10_o_mas', 'sd_y', 'a_hasta_4',
       'a_entre_5_y_8', 'a_entre_9_y_12', 'a_entre_13_y_16', 'a_entre_17_y_19',
       'a_20_o_mas'],
      dtype='object')

In [82]:
#total_invsolvencias

# Crear nueva columna con el total de insolvencias
df_merged["total_insolvencias"] = (
    df_merged["t_voluntario"].fillna(0) +
    df_merged["t_necesario"].fillna(0) +
    df_merged["t_consecutivo"].fillna(0)
)

# Confirmar que se creó correctamente
print(df_merged[["t_voluntario", "t_necesario", "t_consecutivo", "total_insolvencias"]].head())


   t_voluntario  t_necesario  t_consecutivo  total_insolvencias
0           9.0          0.0           13.0                22.0
1          10.0          0.0           21.0                31.0
2          12.0          0.0           17.0                29.0
3          17.0          0.0           21.0                38.0
4          11.0          0.0           12.0                23.0


In [83]:
# ============================
# 3. Crear mapeo provincia → CCAA en df_merged
# ============================
# Diccionario provincia → Comunidad Autónoma (normalizada)
provincia_to_ccaa = {
    # Andalucía
    "Almería": "Andalucia",
    "Cádiz": "Andalucia",
    "Córdoba": "Andalucia",
    "Granada": "Andalucia",
    "Huelva": "Andalucia",
    "Jaén": "Andalucia",
    "Málaga": "Andalucia",
    "Sevilla": "Andalucia",

    # Aragón
    "Huesca": "Aragon",
    "Teruel": "Aragon",
    "Zaragoza": "Aragon",
    "Aragón": "Aragon",

    # Asturias
    "Asturias": "Asturias",

    # Baleares
    "Balears, Illes": "Baleares",

    # Canarias
    "Palmas, Las": "Canarias",
    "Santa Cruz de Tenerife": "Canarias",
    "Canarias": "Canarias",

    # Cantabria
    "Cantabria": "Cantabria",

    # Castilla y León
    "Ávila": "Castilla y Leon",
    "Burgos": "Castilla y Leon",
    "León": "Castilla y Leon",
    "Palencia": "Castilla y Leon",
    "Salamanca": "Castilla y Leon",
    "Segovia": "Castilla y Leon",
    "Soria": "Castilla y Leon",
    "Valladolid": "Castilla y Leon",
    "Zamora": "Castilla y Leon",

    # Castilla-La Mancha
    "Albacete": "Castilla La Mancha",
    "Ciudad Real": "Castilla La Mancha",
    "Cuenca": "Castilla La Mancha",
    "Guadalajara": "Castilla La Mancha",
    "Toledo": "Castilla La Mancha",

    # Cataluña
    "Barcelona": "Cataluña",
    "Girona": "Cataluña",
    "Lleida": "Cataluña",
    "Tarragona": "Cataluña",
    "Cataluña": "Cataluña",

    # Comunitat Valenciana
    "Alicante/Alacant": "Valencia",
    "Castellón/Castelló": "Valencia",
    "Valencia/València": "Valencia",
    "Valencia": "Valencia",

    # Extremadura
    "Badajoz": "Extremadura",
    "Cáceres": "Extremadura",

    # Galicia
    "Coruña, A": "Galicia",
    "Lugo": "Galicia",
    "Ourense": "Galicia",
    "Pontevedra": "Galicia",

    # Madrid
    "Madrid": "Madrid",

    # Murcia
    "Murcia": "Murcia",

    # Navarra
    "Navarra": "Navarra",

    # País Vasco
    "Araba/Álava": "Pais Vasco",
    "Bizkaia": "Pais Vasco",
    "Gipuzkoa": "Pais Vasco",
    "País Vasco": "Pais Vasco",

    # La Rioja
    "Rioja, La": "La Rioja",

    # Ceuta y Melilla
    "Ceuta": "Ceuta",
    "Melilla": "Melilla"
}

# Crear nueva columna en dataset 1 con la comunidad correspondiente
import pandas as pd

df_merged = pd.read_csv("C:/Users/juana/Desktop/liquideuda_project/datasets_clean/merged_registradores.csv")

df_merged["comunidad_autonoma"] = df_merged["comunidad_provincia"].map(provincia_to_ccaa)

# Validar
print(df_merged[["comunidad_provincia", "comunidad_autonoma"]].drop_duplicates().head(20))


        comunidad_provincia comunidad_autonoma
0                   Almería          Andalucia
18                    Cádiz          Andalucia
36                  Córdoba          Andalucia
54                  Granada          Andalucia
72                   Huelva          Andalucia
90                     Jaén          Andalucia
108                  Málaga          Andalucia
126                 Sevilla          Andalucia
144                  Huesca             Aragon
162                  Teruel             Aragon
180                Zaragoza             Aragon
198                Asturias           Asturias
216          Balears, Illes           Baleares
234  Santa Cruz de Tenerife           Canarias
252                Canarias           Canarias
270               Cantabria          Cantabria
288                   Ávila    Castilla y Leon
306                  Burgos    Castilla y Leon
324                    León    Castilla y Leon
342                Palencia    Castilla y Leon


In [84]:
# Convertir la columna año a tipo datetime (formato fecha con solo el año)
df_merged["año"] = pd.to_datetime(df_merged["año"], format="%Y")

# Verificar el cambio
print(df_merged[["año", "año"]].head())
print(df_merged["año"].dtype)


         año        año
0 2021-01-01 2021-01-01
1 2021-01-01 2021-01-01
2 2021-01-01 2021-01-01
3 2021-01-01 2021-01-01
4 2022-01-01 2022-01-01
datetime64[ns]


In [85]:
import os
# Definir ruta destino
ruta_destino = r"C:\Users\juana\Desktop\liquideuda_project\datasets_clean"

# Guardar CSV en la ruta destino
ruta_csv = os.path.join(ruta_destino, "merged_registradores.csv")
df_merged.to_csv(ruta_csv, index=False, encoding="utf-8-sig")

print(f"Archivo guardado en: {ruta_csv}")

Archivo guardado en: C:\Users\juana\Desktop\liquideuda_project\datasets_clean\merged_registradores.csv


In [86]:
df_merged.shape

(882, 58)